In [1]:
# Suppress warnings 
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import numpy as np
import pandas as pd
np.set_printoptions(linewidth=200)
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 100

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="darkgrid")

In [5]:
import math
from numpy.linalg import norm
from scipy import signal, stats
from statsmodels.regression.linear_model import burg

In [204]:
class ObtainFeatures():
    fs = 50  # Sampling frequency of sensor data.
    win_size = 128  # WIndow size of sliding window to segment raw signals.
    overlap_rate = 0.5  # Overlap rate of sliding window to segment raw signals.
    win_sec = 2.56  # Elapsing time (second) of window. 
    
    
    def __init__(self, acc, gyro):
        """
        Args:
            acc (DataFrame): Accelerometer 3-axial raw signals
            gyro (DataFrame): Gyroscope 3-axial raw signals
        """
        self.acc = acc
        self.gyro = gyro
        self.acc_seg = None
        self.gyro_seg = None
        
        
    def apply_median_filter(self, sensor_signal):
        """
        A median filter with window length 5 is applied to remove noise in signals.
        Args:
            sensor_signal (DataFrame): Raw signal
        Returns:
            retval (DataFrame): Median filtered signal
        """
        return sensor_signal.rolling(window=5, center=True, min_periods=1).median()
        
        
    def apply_butterworth_filter(self, sensor_signal):
        """
        A 3rd order low pass Butterworth filter with a corner frequency of 20 Hz is applied to remove noise in signals.
        Args:
            sensor_signal (DataFrame): Raw signal
        Returns:
            retval (DataFrame): Butterworth filtered signal
        """       
        fc = 20  # cutoff frequency
        w = fc / (self.__class__.fs / 2)  # Normalize the frequency
        b, a = signal.butter(3, w, 'low')  # 3rd order low pass Butterworth filter
        
        return pd.DataFrame(signal.filtfilt(b, a, sensor_signal, axis=0), columns=['x', 'y', 'z'])  # Apply Butterworth filter
       
    
    def segment_signal(self, acc, gyro):
        """
        Sample sensor signals in fixed-width sliding windows of 2.56 sec and 50% overlap (128 readings/window).
        Args:
            acc (DataFrame): Raw acceleration signal
            gyro (DataFrame): Raw gyro signal
        Returns:
            acc_seg (list): List of segmented acceleration sigmal. Element is a segmented DataFrame.
            gyro_seg (list): List of segmented gyro sigmal. Element is a segmented DataFrame.
        """
        assert len(acc) == len(gyro), f'Length of acceleration signal ({len(acc)}) does not match to length of gyro signal ({len(gyro)}).'
        
        win_size = self.__class__.win_size
        overlap_rate = self.__class__.overlap_rate
        acc_seg, gyro_seg = [], []
        
        for start_idx in range(0, len(acc) - win_size, int(win_size * overlap_rate)):
            acc_seg.append(acc.iloc[start_idx:start_idx + win_size].reset_index(drop=True))
            gyro_seg.append(gyro.iloc[start_idx:start_idx + win_size].reset_index(drop=True))

        return acc_seg, gyro_seg
        
    
    def remove_gravity(self, acc):
        """
        Separate acceleration signal into body and gravity acceleration signal.
        Another low pass Butterworth filter with a corner frequency of 0.3 Hz is applied.
        Args:
            acc (DataFrame): Segmented acceleration signal
        Returns:
            acc_body (DataFrame): Body acceleration signal
            acc_grav (DataFrame): Gravity acceleration signal
        """
        fc = 0.3  # cutoff frequency
        w = fc / (self.__class__.fs / 2)  # Normalize the frequency
        b, a = signal.butter(3, w, 'low')  # 3rd order low pass Butterworth filter
        
        acc_grav = pd.DataFrame(signal.filtfilt(b, a, acc, axis=0), columns=['x', 'y', 'z'])  # Apply Butterworth filter
        
        # Substract gravity acceleration from acceleration sigal.
        acc_body = acc - acc_grav
        return acc_body, acc_grav
    
    
    def obtain_jerk_signal(self, sensor_signal):
        """
        Derive signal to obtain Jerk signals
        Args:
            sensor_signal (DataFrame)
        Returns:
            jerk_signal (DataFrame):
        """
        jerk_signal = sensor_signal.diff(periods=1)  # Calculate difference 
        jerk_signal.iloc[0] = jerk_signal.iloc[1]  # Fillna
        jerk_signal = jerk_signal / (1 / self.__class__.fs)  # Derive in time (1 / sampling frequency)
        return jerk_signal
    
    
    def obtain_magnitude(self, sensor_signal):
        """
        Calculate the magnitude of these three-dimensional signals using the Euclidean norm
        Args:
            sensor_signal (DataFrame): Three-dimensional signals
        Returns:
            retval (array): Magnitude of three-dimensional signals
        """
        return  norm(sensor_signal, ord=2, axis=1)
    
    
    def obtain_amplitude_spectrum(self, sensor_signal):
        """
        Obtain amplitude spectrum using Fast Fourier Transform (FFT).
        Args:
            sensor_signal (DataFrame): Time domain signals
        Returns:
            amp (array): Amplitude spectrum
        """
        N = len(sensor_signal)
        # dim = sensor_signal.shape[1]
        win = np.hamming(N)  # hamming window
        
        if len(sensor_signal.shape) == 1:
            sensor_signal = sensor_signal * win  # Apply hamming window
            F = np.fft.fft(sensor_signal, axis=0)  # Apply FFT
            F = F[:N//2]  # Remove the overlapping part

            amp = np.abs(F)  # Obtain the amplitude spectrum
            amp = amp / N * 2
            amp[0] = amp[0] / 2
        
        else:
            df = pd.DataFrame(columns=['x', 'y', 'z'])
            for axis in ['x', 'y', 'z']:
                df[axis] = sensor_signal[axis] * win  # Apply hamming window

            F = np.fft.fft(df, axis=0)  # Apply FFT
            F = pd.DataFrame(F, columns=['x', 'y', 'z'])  # Convert array to DataFrame
            F = F[:N//2]  # Remove the overlapping part

            amp = np.abs(F)  # Obtain the amplitude spectrum
            amp = pd.DataFrame(amp, columns=['x', 'y', 'z'])  # Convert array to DataFrame

            amp = amp / N * 2
            amp.iloc[0] = amp.iloc[0] / 2
        
        return amp
    
    
    def obtain_mean(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:    return sensor_signal.mean()
        else:    return sensor_signal.mean().values
    
    def obtain_std(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:    return sensor_signal.std()
        else:    return sensor_signal.std().values
    
    def obtain_mad(self, sensor_signal) -> np.ndarray:
        return stats.median_absolute_deviation(sensor_signal, axis=0)
    
    def obtain_max(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:    return sensor_signal.max()
        else:    return sensor_signal.max().values
    
    def obtain_min(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:    return sensor_signal.min()
        else:    return sensor_signal.min().values
    
    def obtain_sma(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:    return (sensor_signal.sum() - self.obtain_min(sensor_signal) * len(sensor_signal)) / self.__class__.win_sec
        else:    return sum(sensor_signal.sum().values - self.obtain_min(sensor_signal) * len(sensor_signal)) / self.__class__.win_sec
    
    def obtain_energy(self, sensor_signal) -> np.ndarray:
        return  norm(sensor_signal, ord=2, axis=0) ** 2 / len(sensor_signal)
    
    def obtain_iqr(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:
            q25, q75 = np.percentile(sensor_signal, q=[25, 75])
            return q75 - q25
        else:    return  sensor_signal.quantile(.75).values - sensor_signal.quantile(.25).values
    
    def obtain_entropy(self, sensor_signal) -> np.ndarray:
        return  stats.entropy(sensor_signal)

    def obtain_arCoeff(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:  # Signal dimension is 1
            arCoeff, _ = burg(sensor_signal, order=4)
        else:  # Signal dimension is 3
            x, _ = burg(sensor_signal['x'], order=4)
            y, _ = burg(sensor_signal['y'], order=4)
            z, _ = burg(sensor_signal['z'], order=4)
            arCoeff = np.hstack((x, y, z))
        return arCoeff
    
    def obtain_correlation(self, sensor_signal) -> np.ndarray:
         if len(sensor_signal.shape) == 1:  # Signal dimension is 1
            correlation = np.array([])
         else:  # Signal dimension is 3
            xy = np.corrcoef(sensor_signal['x'], sensor_signal['y'])[0][1]
            yz = np.corrcoef(sensor_signal['y'], sensor_signal['z'])[0][1]
            zx = np.corrcoef(sensor_signal['z'], sensor_signal['x'])[0][1]
            correlation = np.hstack((xy, yz, zx))
         return correlation

    def obtain_maxInds(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:    return np.argmax(sensor_signal)
        return sensor_signal.idxmax().values
        
    def obtain_meanFreq(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:  # Signal dimension is 1
            meanFreq = np.mean(sensor_signal * np.arange(len(sensor_signal)))
        else:  # Signal dimension is 3
            x = np.mean(sensor_signal['x'] * sensor_signal.index.values)
            y = np.mean(sensor_signal['y'] * sensor_signal.index.values)
            z = np.mean(sensor_signal['z'] * sensor_signal.index.values)
            meanFreq = np.hstack((x, y, z))
        return meanFreq
    
    def obtain_skewness(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:
            sensor_signal = pd.DataFrame(sensor_signal)
        return sensor_signal.skew().values
    
    def obtain_kurtosis(self, sensor_signal) -> np.ndarray:
        if len(sensor_signal.shape) == 1:
            sensor_signal = pd.DataFrame(sensor_signal)
        return sensor_signal.kurt().values
    
    def obtain_bandsEnergy(self, sensor_signal) -> np.ndarray:
        bandsEnergy = np.array([])
        bins =  [0, 4, 8, 12, 16, 20, 24, 29, 34, 39, 44, 49, 54, 59, 64]
        for i in range(len(bins) - 1):
            df = sensor_signal.iloc[bins[i]:bins[i+1]]
            arr = self.obtain_energy(df)
            bandsEnergy = np.hstack((bandsEnergy, arr))
        return bandsEnergy

    def obtain_angle(self, sensor_signal_1, sensor_signal_2) -> np.ndarray:
        length = lambda v: math.sqrt(np.dot(v, v))
        return math.acos(np.dot(v1, v2) / (length(v1) * length(v2)))
                
            
    def estimate_variables(self, sensor_signal, domain='t'):
        """
        Calculate statistic variables from signal.
        Args:
            sensor_signal (DataFrame)
            domain (str): Domain of x axis: 't' indicates time domain, 'f' indicates frequency domain.
        Returns:
            variables (array):
                mean(): Mean value
                std(): Standard deviation
                mad(): Median absolute deviation 
                max(): Largest value in array
                min(): Smallest value in array
                sma(): Signal magnitude area
                energy(): Energy measure. Sum of the squares divided by the number of values. 
                iqr(): Interquartile range 
                entropy(): Signal entropy
                arCoeff(): Autorregresion coefficients with Burg order equal to 4
                correlation(): correlation coefficient between two signals
                maxInds(): index of the frequency component with largest magnitude
                meanFreq(): Weighted average of the frequency components to obtain a mean frequency
                skewness(): skewness of the frequency domain signal 
                kurtosis(): kurtosis of the frequency domain signal 
                bandsEnergy(): Energy of a frequency interval within the 64 bins of the FFT of each window.
                angle(): Angle between two vectors. These are used on the angle() variable:
                    gravityMean
                    tBodyAccMean
                    tBodyAccJerkMean
                    tBodyGyroMean
                    tBodyGyroJerkMean
    """

In [205]:
# import logging
# Create log file
# EXEC_TIME = 'create-features-' + datetime.now().strftime("%Y%m%d-%H%M%S")
# logging.basicConfig(filename=f'../../logs/{EXEC_TIME}.log', level=logging.DEBUG)
# logging.debug(f'../../logs/{EXEC_TIME}.log')

In [219]:
def create_features():
    # logging.debug('Start creating features')
    root = '../data/hapt_data_set/RawData/'
    acc_raw = pd.read_table(root + 'acc_exp01_user01.txt', sep=' ', header=None, names=['x', 'y', 'z'])
    gyro_raw = pd.read_table(root + 'gyro_exp01_user01.txt', sep=' ', header=None, names=['x', 'y', 'z'])
    print(f'Acceleration raw signal shape {acc_raw.shape}')
    print(f'Gyroscople raw signal shape {gyro_raw.shape}')
    # logging.debug(f'Acceleration raw signal shape {acc_raw.shape}')
    # logging.debug(f'Gyroscople raw signal shape {gyro_raw.shape}')
    
    of = ObtainFeatures(acc_raw, gyro_raw)
    
    # Remove noises by median filter & Butterworth filter
    print('Remove noises by median filter & Butterworth filter')
    # logging.debug('Remove noises by median filter & Butterworth filter')
    acc_raw = of.apply_median_filter(acc_raw)
    acc_raw = of.apply_butterworth_filter(acc_raw)
    
    gyro_raw = of.apply_median_filter(gyro_raw)
    gyro_raw = of.apply_butterworth_filter(gyro_raw)
    
    
    # Sample signals in fixed-width sliding windows
    print('Sample signals in fixed-width sliding windows')
    # logging.debug('Sample signals in fixed-width sliding windows')
    tAccXYZ, tBodyGyroXYZ = of.segment_signal(acc_raw, gyro_raw)
    print(f'tAcc-XYZ length {len(tAccXYZ)}, tGyro-XYZ length {len(tBodyGyroXYZ)}')
    # print(acc_seg[0].shape)
    # logging.debug(f'tAcc-XYZ length {len(tAccXYZ)}, tGyro-XYZ length {len(tBodyGyroXYZ)}')
    
    
    # Separate acceleration signal into body and gravity acceleration signal
    print('Separate acceleration signal into body and gravity acceleration signal')
    # logging.debug('Separate acceleration signal into body and gravity acceleration signal')
    tBodyAccXYZ, tGravityAccXYZ = [], []
    for acc in tAccXYZ:
        body_acc, grav_acc = of.remove_gravity(acc)
        tBodyAccXYZ.append(body_acc)
        tGravityAccXYZ.append(grav_acc)
        
    print(f'tBodyAcc-XYZ length {len(tBodyAccXYZ)}, tGravityAcc-XYZ length {len(tGravityAccXYZ)}')
    # print(acc_body_seg[0].shape)
    # logging.debug(f'tBodyAcc-XYZ length {len(tBodyAccXYZ)}, tGravityAcc-XYZ length {len(tGravityAccXYZ)}')
    
    
    # Obtain Jerk signals of body linear acceleration and angular velocity
    print('Obtain Jerk signals of body linear acceleration and angular velocity')
    # logging.debug('Obtain Jerk signals of body linear acceleration and angular velocity')
    tBodyAccJerkXYZ, tBodyGyroJerkXYZ = [], []
    for body_acc, gyro in zip(tBodyAccXYZ, tBodyGyroXYZ):
        body_acc_jerk = of.obtain_jerk_signal(body_acc)
        gyro_jerk = of.obtain_jerk_signal(gyro)
        
        tBodyAccJerkXYZ.append(body_acc_jerk)
        tBodyGyroJerkXYZ.append(gyro_jerk)
        
    print(f'tBodyAccJerk-XYZ length {len(tBodyAccJerkXYZ)}, tBodyGyroJerk-XYZ length {len(tBodyGyroJerkXYZ)}')
    # print(acc_body_jerk_seg[0].shape)
    # logging.debug(f'tBodyAccJerk-XYZ length {len(tBodyAccJerkXYZ)}, tBodyGyroJerk-XYZ length {len(tBodyGyroJerkXYZ)}')
    
    
    # Calculate the magnitude of three-dimensional signals using the Euclidean norm
    print('Calculate the magnitude of three-dimensional signals using the Euclidean norm')
    # logging.debug('Calculate the magnitude of three-dimensional signals using the Euclidean norm')
    tBodyAccMag, tGravityAccMag, tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag = [], [], [], [], []
    for body_acc, grav_acc, body_acc_jerk, gyro, gyro_jerk in zip(tBodyAccXYZ, tGravityAccXYZ, tBodyAccJerkXYZ, tBodyGyroXYZ, tBodyGyroJerkXYZ):
        body_acc_mag = of.obtain_magnitude(body_acc)
        grav_acc_mag = of.obtain_magnitude(grav_acc)
        body_acc_jerk_mag = of.obtain_magnitude(body_acc_jerk)
        gyro_mag = of.obtain_magnitude(gyro)
        gyro_jerk_mag = of.obtain_magnitude(gyro_jerk)
        
        tBodyAccMag.append(body_acc_mag)
        tGravityAccMag.append(grav_acc_mag)
        tBodyAccJerkMag.append(body_acc_jerk_mag)
        tBodyGyroMag.append(gyro_mag)
        tBodyGyroJerkMag.append(gyro_jerk_mag)
        
    
    # Obtain amplitude spectrum using Fast Fourier Transform (FFT).
    print('Obtain amplitude spectrum using Fast Fourier Transform (FFT).')
    # logging.debug('Obtain amplitude spectrum using Fast Fourier Transform (FFT).')
    fBodyAccXYZ, fBodyAccJerkXYZ, fBodyGyroXYZ, fBodyAccMag, fBodyAccJerkMag, fBodyGyroMag, fBodyGyroJerkMag = [], [], [] ,[] ,[] ,[], []
    for body_acc, body_acc_jerk, gyro, body_acc_mag, body_acc_jerk_mag, gyro_mag, gyro_jerk_mag in \
        zip(tBodyAccXYZ, tBodyAccJerkXYZ, tBodyGyroXYZ, tBodyAccMag, tBodyAccJerkMag, tBodyGyroMag, tBodyGyroJerkMag):
        body_acc_amp = of.obtain_amplitude_spectrum(body_acc)
        body_acc_jerk_amp = of.obtain_amplitude_spectrum(body_acc_jerk)
        gyro_amp = of.obtain_amplitude_spectrum(gyro)
        body_acc_mag_amp =  of.obtain_amplitude_spectrum(body_acc_mag)
        body_acc_jerk_mag_amp = of.obtain_amplitude_spectrum(body_acc_jerk_mag)
        gyro_mag_amp = of.obtain_amplitude_spectrum(gyro_mag)
        gyro_jerk_mag_amp = of.obtain_amplitude_spectrum(gyro_jerk_mag)
        
        fBodyAccXYZ.append(body_acc_amp)
        fBodyAccJerkXYZ.append(body_acc_jerk_amp)
        fBodyGyroXYZ.append(gyro_amp)
        fBodyAccMag.append(body_acc_mag_amp)
        fBodyAccJerkMag.append(body_acc_jerk_mag_amp)
        fBodyGyroMag.append(gyro_mag_amp)
        fBodyGyroJerkMag.append(gyro_jerk_mag_amp)
    
    
    #  Following signals are obtained.
    time_signals = [
        tBodyAccXYZ, 
        tGravityAccXYZ, 
        tBodyAccJerkXYZ, 
        tBodyGyroXYZ, 
        tBodyGyroJerkXYZ,
        tBodyAccMag,
        tGravityAccMag, 
        tBodyAccJerkMag, 
        tBodyGyroMag, 
        tBodyGyroJerkMag]
    freq_signals = [
        fBodyAccXYZ,
        fBodyAccJerkXYZ,
        fBodyGyroXYZ,
        fBodyAccMag,
        fBodyAccJerkMag,
        fBodyGyroMag,
        fBodyGyroJerkMag]
    all_signals = time_signals + freq_signals
    
    
    # Calculate feature vectors
    print('Calculate feature vectors')
    # logging.debug('Calculate feature vectors')
    features = np.zeros((len(tBodyAccXYZ), 561))
    
    for i in range(len(tBodyAccXYZ)):
        feature_vector = np.array([])
        
        # mean, std, mad, max, min, sma, energy, iqr, entropy
        for t_signal in all_signals:
            sig = t_signal[i]
            mean = of.obtain_mean(sig)
            std = of.obtain_std(sig)
            mad = of.obtain_mad(sig)
            max_val = of.obtain_max(sig)
            min_val = of.obtain_min(sig)
            sma = of.obtain_sma(sig)
            energy =  of.obtain_energy(sig)
            iqr = of.obtain_iqr(sig)
            entropy = of.obtain_entropy(sig)
            print(mean, std, mad, max_val, min_val, sma, energy, iqr, entropy)
            return
            
            feature_vector = np.hstack((feature_vector, mean, std, mad, max_val, min_val, sma, energy, iqr, entropy))
        
        # arCoeff
        for t_signal in time_signals:
            sig = t_signal[i]
            arCoeff = of.obtain_arCoeff(sig)
            
            feature_vector = np.hstack((feature_vector, arCoeff))
        
        # correlation
        for t_signal in [tBodyAccXYZ, tGravityAccXYZ, tBodyAccJerkXYZ, tBodyGyroXYZ, tBodyGyroJerkXYZ]:
            sig = t_signal[i]
            correlation = of.obtain_correlation(sig)
            
            feature_vector = np.hstack((feature_vector, correlation))
        
        # maxInds, meanFreq, skewness, kurtosis
        for t_signal in freq_signals:
            sig = t_signal[i]
            maxInds = of.obtain_maxInds(sig)
            meanFreq = of.obtain_meanFreq(sig)
            skewness = of.obtain_skewness(sig)
            kurtosis = of.obtain_kurtosis(sig)
            
            feature_vector = np.hstack((feature_vector, maxInds, meanFreq, skewness, kurtosis))
        
        # bandsEnergy
        for t_signal in [tBodyAccXYZ, tBodyAccJerkXYZ, tBodyGyroXYZ]:
            sig = t_signal[i]
            bandsEnergy = of.obtain_bandsEnergy(sig)
            
            feature_vector = np.hstack((feature_vector, bandsEnergy))
        
        # angle
        gravityMean = tGravityAccXYZ[i].mean()
        tBodyAccMean = tBodyAccXYZ[i].mean()
        tBodyAccJerkMean = tBodyAccJerkXYZ[i].mean()
        tBodyGyroMean = tBodyGyroXYZ[i].mean()
        tBodyGyroJerkMean = tBodyGyroJerkXYZ[i].mean()
        tXAxisAcc = tAccXYZ[i]['x']
        tXAxisGravity = tGravityAccXYZ[i]['x']
        tYAxisAcc = tAccXYZ[i]['y']
        tYAxisGravity = tGravityAccXYZ[i]['y']
        tZAxisAcc = tAccXYZ[i]['z']
        tZAxisGravity = tGravityAccXYZ[i]['z']
        
        
        tBodyAccWRTGravity = of.obtain_angle(tBodyAccMean, gravityMean)
        tBodyAccJerkWRTGravity = of.obtain_angle(tBodyAccJerkMean, gravityMean)
        tBodyGyroWRTGravity = of.obtain_angle(tBodyGyroMean, gravityMean)
        tBodyGyroJerkWRTGravity = of.obtain_angle(tBodyGyroJerkMean, gravityMean)
        tXAxisAccWRTGravity = of.obtain_angle(tXAxisAcc, tXAxisGravity)
        tYAxisAccWRTGravity = of.obtain_angle(tYAxisAcc, tYAxisGravity)
        tZAxisAccWRTGravity = of.obtain_angle(tZAxisAcc, tZAxisGravity)
        
        feature_vector = np.hstack((feature_vector, tBodyAccWRTGravity, tBodyAccJerkWRTGravity, tBodyGyroWRTGravity, tBodyGyroJerkWRTGravity, \
                                   tXAxisAccWRTGravity, tYAxisAccWRTGravity, tZAxisAccWRTGravity))
    
        print(feature_vector.shape)
        return feature_vector
    
    
    
    return fBodyGyroJerkMag

In [220]:
arr = create_features()

Acceleration raw signal shape (20598, 3)
Gyroscople raw signal shape (20598, 3)
Remove noises by median filter & Butterworth filter
Sample signals in fixed-width sliding windows
tAcc-XYZ length 320, tGyro-XYZ length 320
Separate acceleration signal into body and gravity acceleration signal
tBodyAcc-XYZ length 320, tGravityAcc-XYZ length 320
Obtain Jerk signals of body linear acceleration and angular velocity
tBodyAccJerk-XYZ length 320, tBodyGyroJerk-XYZ length 320
Calculate the magnitude of three-dimensional signals using the Euclidean norm
Obtain amplitude spectrum using Fast Fourier Transform (FFT).
Calculate feature vectors
[ 0.00713712 -0.00565229 -0.06150826] [0.08354449 0.04013641 0.21405018] [0.05096648 0.02525884 0.19456909] [0.34050599 0.08830477 0.37080311] [-0.16827838 -0.10629406 -0.5130356 ] 36.37923137871427 [0.00697609 0.00163029 0.0492428 ] [0.08541761 0.04480894 0.34987618] [-inf -inf -inf]


In [209]:
arr.argmin()

22

In [212]:
arr[22:26]

array([      -inf,       -inf,       -inf, 0.89639311])

In [198]:
arr = np.array((0,1,2,3,4,3,2,1,3,4,))

In [199]:
pd.DataFrame(arr).skew()

0   -0.33436
dtype: float64

In [159]:
obtain_sma(arr)

TypeError: 'numpy.int64' object is not iterable